In [1]:
%pylab inline
import pandas as pd
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
import gluonnlp as nlp
import numpy as np
from torch.utils.data import Dataset, DataLoader
from kogpt2.pytorch_kogpt2 import get_pytorch_kogpt2_model
from gluonnlp.data import SentencepieceTokenizer
from kogpt2.utils import get_tokenizer
from tqdm import tqdm, tqdm_notebook
from transformers import AdamW
from transformers.optimization import WarmupLinearSchedule

tok_path = get_tokenizer()
gpt, vocab = get_pytorch_kogpt2_model()
tok = SentencepieceTokenizer(tok_path,  num_best=0, alpha=0)

Populating the interactive namespace from numpy and matplotlib
using cached model
using cached model
using cached model


In [2]:
import random as rd

In [3]:
#device = torch.device("cuda:0")
device = torch.device("cpu")

In [4]:
dataset = pd.read_csv('C:/Users/mskim3/Desktop/식단인식_알고리즘/data/dataset.csv',encoding='cp949').iloc[:,1:]

In [5]:
class GPT2summarizer(nn.Module):
    def __init__(self, gpt):
        super(GPT2summarizer, self).__init__()
        self.gpt = gpt

    def forward(self, ids):
        out = self.gpt(ids)

        return out

In [6]:
model = GPT2summarizer(gpt).to(device)
model.load_state_dict(torch.load('C:/Users/mskim3/Desktop/식단인식_알고리즘/model/GPT2_Food2.pth', map_location=device))
model.eval()

GPT2summarizer(
  (gpt): GPT2LMHeadModel(
    (transformer): GPT2Model(
      (wte): Embedding(50000, 768)
      (wpe): Embedding(1024, 768)
      (drop): Dropout(p=0.1, inplace=False)
      (h): ModuleList(
        (0): Block(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): Attention(
            (c_attn): Conv1D()
            (c_proj): Conv1D()
            (attn_dropout): Dropout(p=0.1, inplace=False)
            (resid_dropout): Dropout(p=0.1, inplace=False)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): MLP(
            (c_fc): Conv1D()
            (c_proj): Conv1D()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (1): Block(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): Attention(
            (c_attn): Conv1D()
            (c_proj): Conv1D()
            (attn_dropout): Dropout(p=0.1, inplace=False)
          

In [22]:
def exampler():
    sent = dataset['sentence'].iloc[rd.choice(range(10000))] 
    toked = tok(sent)
    linp = [vocab[vocab.bos_token],]  + vocab[toked] + [vocab[vocab.eos_token]] + [vocab['<sys>']]
    inp = torch.tensor(linp)
    i=0
    while i < 50:
        pred = model(inp)[0]
        predi = torch.argmax(pred[-1]).item()
        if predi == vocab[vocab.eos_token]:
            break
        linp = linp + [predi]
        inp = torch.tensor(linp)
    #finalsent = vocab.to_tokens(linp)
    foodlist=[]
    np.where(np.array(linp)==7)
    np.where(np.array(linp)==8)
    for i in range(len(np.where(np.array(linp)==7)[0])):
        si = np.where(np.array(linp)==7)[0][i]
        ei = np.where(np.array(linp)==8)[0][i]
        foodlist.append(''.join(vocab.to_tokens(linp[si+1:ei])).replace('▁',' ').strip())
    print('입력: ', sent)
    print('먹은 음식: ', unique(foodlist).tolist())

In [19]:
def foodextractor(sent):
    toked = tok(sent)
    linp = [vocab[vocab.bos_token],]  + vocab[toked] + [vocab[vocab.eos_token]] + [vocab['<sys>']]
    inp = torch.tensor(linp)
    i=0
    while i < 50:
        pred = model(inp)[0]
        predi = torch.argmax(pred[-1]).item()
        if predi == vocab[vocab.eos_token]:
            break
        linp = linp + [predi]
        inp = torch.tensor(linp)
    #finalsent = vocab.to_tokens(linp)
    foodlist=[]
    np.where(np.array(linp)==7)
    np.where(np.array(linp)==8)
    for i in range(len(np.where(np.array(linp)==7)[0])):
        si = np.where(np.array(linp)==7)[0][i]
        ei = np.where(np.array(linp)==8)[0][i]
        foodlist.append(''.join(vocab.to_tokens(linp[si+1:ei])).replace('▁',' ').strip())
    print('입력: ', sent)
    print('먹은 음식: ', unique(foodlist).tolist())

In [128]:
exampler()

입력:  일일 오후에 수육하고 족발을 적정량 먹을게
먹은 음식:  ['수육', '족발']


In [122]:
sent = input()
foodextractor(sent)

오늘 점심은 총각김치랑 된장찌개 쌀밥이야
입력:  오늘 점심은 총각김치랑 된장찌개 쌀밥이야
먹은 음식:  ['된장찌개', '쌀밥', '총각 김치']


In [28]:
import konlpy
from konlpy.tag import Okt

In [29]:
okt = Okt()

In [63]:
inp = input()
okt.pos(inp)

참치회덮밥


[('참치', 'Noun'), ('회덮밥', 'Noun')]